In [1]:
import pandas as pd
!pip install transformers

     |████████████████████████████████| 4.2 MB 5.4 MB/s 
     |████████████████████████████████| 6.6 MB 55.8 MB/s 
     |████████████████████████████████| 596 kB 60.7 MB/s 
     |████████████████████████████████| 84 kB 3.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
link = 'https://drive.google.com/file/d/19GBWIsFGmfQyyCe4YRE0zSoETnhOqmPj/view?usp=sharing' 

In [5]:
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Train_Feb_VADER.csv') 
 
df = pd.read_csv('Train_Feb_VADER.csv')
print(df)

         Unnamed: 0                                               text  \
0                 0   Brother this called link 👇\n\n\nIt's an advis...   
1                 1  Ukraine invasion: Who is President Volodymyr Z...   
2                 2  So it now looks like we're in a new phase of t...   
3                 3    TOP RUSSIAN AGGRESSION AGAINST #UKRAINE.  CL...   
4                 4    Ukraine won't fall for Russia because the Uk...   
...             ...                                                ...   
1782546     1782546   Dear UNESCO. I'm a peaceful citizen of Ukrain...   
1782547     1782547  This is not Ukraine vs Russia war. This is a c...   
1782548     1782548  Russia 🇷🇺 is warn... Don't provoke the 4nation...   
1782549     1782549  Africa Report: How Africa factors in #RussiaUk...   
1782550     1782550     I live in the west..  I'm Indian... agreed ...   

         Compound  Positive  Negative  Neutral Sentiment  
0         -0.3063     0.055     0.104    0.841  Nega

In [6]:
df

,Unnamed: 0,text,Compound,Positive,Negative,Neutral,Sentiment
0,0,Brother this called link 👇\n\n\nIt's an advis...,-0.3063,0.055,0.104,0.841,Negative
1,1,Ukraine invasion: Who is President Volodymyr Z...,0.0772,0.085,0.000,0.915,Positive
2,2,So it now looks like we're in a new phase of t...,-0.0000,0.111,0.129,0.760,Neutral
3,3,TOP RUSSIAN AGGRESSION AGAINST #UKRAINE. CL...,0.1739,0.233,0.208,0.559,Positive
4,4,Ukraine won't fall for Russia because the Uk...,0.3818,0.167,0.000,0.833,Positive
...,...,...,...,...,...,...,...
1782546,1782546,Dear UNESCO. I'm a peaceful citizen of Ukrain...,-0.4019,0.157,0.204,0.639,Negative
1782547,1782547,This is not Ukraine vs Russia war. This is a c...,-0.8779,0.087,0.257,0.655,Negative
1782548,1782548,Russia 🇷🇺 is warn... Don't provoke the 4nation...,0.5801,0.239,0.130,0.631,Positive
1782549,1782549,Africa Report: How Africa factors in #RussiaUk...,-0.5994,0.000,0.218,0.782,Negative


In [7]:
df = df.drop(['Unnamed: 0','Compound', 'Positive','Negative','Neutral'], axis=1)

In [8]:
df = df.replace('Negative', 0)
df = df.replace('Neutral', 1)
df = df.replace('Positive', 2)

In [9]:
df

,text,Sentiment
0,Brother this called link 👇\n\n\nIt's an advis...,0
1,Ukraine invasion: Who is President Volodymyr Z...,2
2,So it now looks like we're in a new phase of t...,1
3,TOP RUSSIAN AGGRESSION AGAINST #UKRAINE. CL...,2
4,Ukraine won't fall for Russia because the Uk...,2
...,...,...
1782546,Dear UNESCO. I'm a peaceful citizen of Ukrain...,0
1782547,This is not Ukraine vs Russia war. This is a c...,0
1782548,Russia 🇷🇺 is warn... Don't provoke the 4nation...,2
1782549,Africa Report: How Africa factors in #RussiaUk...,0


In [10]:
#80,10,10 split
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=1)

test_set, valid_set = train_test_split(test_set, test_size=0.5, random_state=1)

In [11]:
#train_set = train_set.sample(frac=0.8,random_state=1)

In [12]:
#valid_set = valid_set.sample(frac=0.8,random_state=1)

In [13]:
#test_set = test_set.sample(frac=0.8,random_state=1)

In [14]:
from google.colab import drive
drive.mount('drive')
import sys
sys.path.append('drive/My Drive/Colab Notebooks')

Mounted at drive


In [18]:
#code from NLPiation

import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load the BERT Tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-large-cased')

# The dataset class
class TheDataset(torch.utils.data.Dataset):

    def __init__(self, reviews, sentiments, tokenizer):
        self.reviews    = reviews
        self.sentiments = sentiments
        self.tokenizer  = tokenizer
        self.max_len    = tokenizer.model_max_length
  
    def __len__(self):
        return len(self.reviews)
  
    def __getitem__(self, index):
        review = str(self.reviews[index])
        sentiments = self.sentiments[index]

        encoded_review = self.tokenizer.encode_plus(
            review,
            add_special_tokens    = True,
            max_length            = 128,
            return_token_type_ids = False,
            return_attention_mask = True,
            return_tensors        = 'pt',
            padding               = "max_length",
            truncation            = True
        )

        return {
            'input_ids': encoded_review['input_ids'][0],
            'attention_mask': encoded_review['attention_mask'][0],
            'labels': torch.tensor(sentiments, dtype=torch.long)
        }

# Prepare the Train/Validation sets
train_set_dataset = TheDataset(
    reviews    = train_set.text.tolist(),
    sentiments = train_set.Sentiment.tolist(),
    tokenizer  = tokenizer,
)

valid_set_dataset = TheDataset(
    reviews    = valid_set.text.tolist(),
    sentiments = valid_set.Sentiment.tolist(),
    tokenizer  = tokenizer,
)

# Load the BERT model
model = BertForSequenceClassification.from_pretrained('bert-large-cased', problem_type="single_label_classification", num_labels=3)

# Freeze the whole BERT model and train just the classifier
#for name, param in model.bert.named_parameters():
    #param.requires_grad = False

# Freeze BERT except the pooler layer
#for name, param in model.bert.named_parameters():
    #if not name.startswith('pooler'):
        #param.requires_grad = False

# Freeze the first 18 layers of the BERT
for name, param in model.bert.named_parameters():
    if ( not name.startswith('pooler') ) and "layer.23" and "layer.22" and "layer.21" and "layer.20" and "layer.19" and "layer.18" not in name :
        param.requires_grad = False

# The function to get the accuracy
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define the training parameters
training_args = TrainingArguments(
    output_dir                  = 'drive/My Drive/Colab Notebooks',
    num_train_epochs            = 8,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size  = 64,
    warmup_steps                = 10000,
    learning_rate               = 5e-5,
    weight_decay                = 0.01,
    adam_beta1                  = 0.9,
    adam_beta2                  = 0.999,
    save_strategy               = "epoch",
    evaluation_strategy         = "epoch",
    eval_steps                  = 1,
    #save_total_limit            = 5,
    #metric_for_best_model       = 'eval_loss',
    load_best_model_at_end      =True
)

# Define the Huggingface Trainer object
trainer = Trainer(
    model           = model,
    args            = training_args,
    train_dataset   = train_set_dataset,
    eval_dataset    = valid_set_dataset,
    compute_metrics = compute_metrics,
    callbacks       = [EarlyStoppingCallback(early_stopping_patience=3)]
)

# Start pre-training!
trainer.train()

loading configuration file https://huggingface.co/bert-large-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/11ad22b0deaa199d15b331609ca5f60872a1a91473e9b40c115192dadb6d9a30.bdf0177a774dcff07681b2527b926c099e6563687c75a79f7469c7a7da7898c7
Model config BertConfig {
  "_name_or_path": "bert-large-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.965300,1.042595,0.538080,0.233226,0.179360,0.333333


***** Running Evaluation *****
  Num examples = 178256
  Batch size = 64
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to drive/My Drive/Colab Notebooks/checkpoint-89128
Configuration saved in drive/My Drive/Colab Notebooks/checkpoint-89128/config.json
Model weights saved in drive/My Drive/Colab Notebooks/checkpoint-89128/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from drive/My Drive/Colab Notebooks/checkpoint-89128 (score: 1.0425950288772583).


TrainOutput(global_step=89128, training_loss=0.9065291380156799, metrics={'train_runtime': 67703.7198, 'train_samples_per_second': 21.063, 'train_steps_per_second': 1.316, 'total_flos': 3.322439729187533e+17, 'train_loss': 0.9065291380156799, 'epoch': 1.0})

In [ ]:
# Load the checkpoint
model = BertForSequenceClassification.from_pretrained("drive/My Drive/Colab Notebooks/epoch 2", num_labels=3)

# Make the test set ready
test_set_dataset = TheDataset(
    reviews    = test_set.text.tolist(),
    sentiments = test_set.Sentiment.tolist(),
    tokenizer  = tokenizer,
)

training_args = TrainingArguments(
    output_dir = 'drive/My Drive/Colab Notebooks',
    do_predict = True
)

trainer = Trainer(
    model           = model,
    args            = training_args,
    compute_metrics =compute_metrics,
)

PredictionOutput = trainer.predict(test_set_dataset)

loading configuration file drive/My Drive/Colab Notebooks/epoch 2/config.json
Model config BertConfig {
  "_name_or_path": "bert-large-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.19.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weight

In [ ]:
import numpy as np
PredictionOutput

PredictionOutput(predictions=array([[ 5.178012 , -2.6322966, -2.3268027],
       [-1.6303229, -2.1019204,  4.0263376],
       [ 5.634117 , -2.888948 , -2.671971 ],
       ...,
       [ 5.5643535, -2.502309 , -3.1386096],
       [ 5.7115645, -2.9520373, -2.1513696],
       [ 5.6934423, -2.8060687, -2.5102293]], dtype=float32), label_ids=array([0, 2, 0, ..., 0, 0, 0]), metrics={'test_loss': 0.19550761580467224, 'test_accuracy': 0.9358615466606828, 'test_f1': 0.9252227195619062, 'test_precision': 0.9356617917872483, 'test_recall': 0.9164653472496113, 'test_runtime': 2501.218, 'test_samples_per_second': 71.267, 'test_steps_per_second': 8.908})

In [ ]:
predictions, label_ids, metrics = PredictionOutput

In [ ]:
metrics

{'test_balanced_accuracy': 0.7978215164846297,
 'test_f1': 0.7996179457912057,
 'test_loss': 0.5070290565490723,
 'test_precision': 0.8017353967317038,
 'test_recall': 0.7978215164846297,
 'test_runtime': 1147.1624,
 'test_samples_per_second': 9.787,
 'test_steps_per_second': 1.224}

In [ ]:
df1 = pd.DataFrame(metrics, index=['Output'])

In [ ]:
df1

,test_loss,test_balanced_accuracy,test_f1,test_precision,test_recall,test_runtime,test_samples_per_second,test_steps_per_second
Output,0.507029,0.797822,0.799618,0.801735,0.797822,1147.1624,9.787,1.224


In [ ]:
df2 = pd.DataFrame(predictions)

In [ ]:
df2

,0,1,2
0,-2.096037,1.464743,0.665169
1,3.632438,-2.071273,-2.919343
2,0.936616,-1.865878,0.049600
3,1.543650,-1.613585,-0.860745
4,-3.516509,4.259713,-1.870149
...,...,...,...
11222,-1.398229,-1.450295,2.285965
11223,-3.191445,-3.537298,7.063457
11224,2.085979,-1.913138,-0.325679
11225,5.274303,-3.780576,-1.656197


In [ ]:
test_set

,text,Sentiment
43963,Biden admin is handing Ukraine over to Russia ...,2
41962,We know Trump held Putin at bay because Russi...,0
20660,Russia is not a credible threat to our No ...,0
45182,The more NATO moves their fence closer to Rus...,0
696,"Putin-Biden video call. 3 reasons why ""Russia ...",1
...,...,...
40518,What about Ukraine and Russia's other neighbo...,2
18759,"If Russia invades Ukraine, good luck - your o...",2
16105,"Dr. David Donat Cattin, Secretary-General of ,...",0
35183,russia: US warns Russia of crippling sanctions...,0
